# Loading of Datasets

Based on the preprocessing conducted, we have selected the following datasets to conduct an evaluation to determine which dataset will be used before performing hyperparameter tuning on our desired dataset.

The datasets that we have chosen are:
- `noncir_ss_scaled_trimmed_cir_pca_ss_scaled.pkl`: Non-Cir (Standard Scaled after feature selection) with CIR Statistical Measures (PCA and Standard Scaled)
- `noncir_ss_scaled_trimmed_cir_ss_scaled.pkl`: Non-CIR (Standard Scaled after feature selection) with CIR Statistical Measures (Standard Scaled)
- `noncir_ss_scaled_trimmed_cir_pca.pkl`: Non-CIR (Standard Scaled after feature selection) with CIR Statistical Measures (PCA)

The datasets will be loaded into `dataset_1`, `dataset_2`, `dataset_3` respectively.

In [4]:
from utils import *
import pandas as pd
import numpy as np

dataset_1 = load_from_pickle("noncir_ss_scaled_trimmed_cir_pca_ss_scaled.pkl")
dataset_2 = load_from_pickle("noncir_ss_scaled_trimmed_cir_ss_scaled.pkl")
dataset_3 = load_from_pickle("noncir_ss_scaled_trimmed_cir_pca.pkl")

# Multi-Layer Perceptron (MLP)

## Dataset 1 (noncir_ss_scaled_trimmed_cir_pca_ss_scaled)

Evaluation of the performance of `dataset_1`. With a 70:30 training and test split

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

# Select the features to be used for Support Vector Classification
X = dataset_1.drop(columns = 'NLOS')
Y = dataset_1[['NLOS']].to_numpy()
Y = Y.reshape(-1)

# Standardizing the data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)

# Split dataset into 70% training and 30% test
x_train, x_test, y_train, y_test = train_test_split(x_scaled, Y, test_size = TEST_SIZE, random_state = RANDOM_STATE)

### Hyperparameter Tuning for MLPClassifier

The hyperparameter tuning helps to test all the various combinations that we have defined within the parameter space and provides the parameters that provides the most optimal results.

To determine the most optimal parameters for the MLP Classifier, we will conduct Hyperparameter tuning using GridSearchCV.

The following are the parameters that we will be tuning:
- `hidden_layer_sizes`
    - Determines the number of neurons in the ith hidden layer (E.g. (10, 10) Represents 10 neurons in 2 hidden layers)
- `activation`
    - Activation function for the hidden layer
        - `tanh` Hyperbolic tan function, returns f(x) = tanh(x)
        - `relu` Rectified linear unit function, returns f(x) = max(0, x)
- `solver`
    - Solver for weight optimization
        - `adam` Stochastic gradient-based optimiser proposed by Kingma, Diederik, and Jimmy Ba
        - `sgd` Stochastic gradient descent
- `alpha`
    - Strength of the L2 regularization term aka penalty term, that combats overfitting by constraining the size of the weights. Increasing alpha may fix high variance (a sign of overfitting) by encouraging smaller weights, resulting in a decision boundary plot that appears with lesser curvatures. Similarly, decreasing alpha may fix high bias (a sign of underfitting) by encouraging larger weights, potentially resulting in a more complicated decision boundary.
- `learning_rate`
    - Learning rate schedule for weight updates
        - `constant` Constant learning rate given by default which is 0.001
        - `adaptive` Keeps the learning rate constant to default 0.001 if the training loss keeps decreasing but for every 2 consecutive epochs fail to decrease the training loss, the current learning rate is divided by 5

In [2]:
mlp = MLPClassifier(max_iter = 1000)

parameter_space = {
    'hidden_layer_sizes': [(50), (50, 50), (50, 50, 50), (50, 100, 50), (100, 100, 100), (50, 50 ,50), (50, 50, 50, 50), (100, 100, 100, 100), (100, 100), (100)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive']
}

NameError: name 'MLPClassifier' is not defined

The GridSearchCV will be configured with the following parameters:
- `cv`
    - Cross Validation spliting strategy where the value determine the number of folds.
- `n_jobs`
    - Number of jobs to run in parallel (Determines how many processors to use).

In [7]:
from sklearn.model_selection import GridSearchCV

# CV (Cross Validation) helps to split our dataset into random groups and have 1 group being the test dataset whilst the rest are used for training - Helps to reduce overfitting
clf = GridSearchCV(mlp, parameter_space, n_jobs = 1, cv = 3, verbose = 3)
clf.fit(x_train, y_train)

/Users/cezaartan/Desktop/University/SIT/CSC3105 - Data Analytics/CSC3105-DA-T2/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cezaartan/Desktop/University/SIT/CSC3105 - Data Analytics/CSC3105-DA-T2/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cezaartan/Desktop/University/SIT/CSC3105 - Data Analytics/CSC3105-DA-T2/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cezaartan/Desktop/University/SIT/CSC3105 - Data Analytics/CSC3105-DA-T2/venv/lib/python3.9

In [ ]:
print("Best Parameters Found: \n", clf.best_params_)

# print how our model looks after hyper-parameter tuning 
print(clf.best_estimator_) 

In [ ]:
# Displays the mean, sd and paremeters of the training scores
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
clf = MLPClassifier(activation = 'relu', solver = 'adam', hidden_layer_sizes = (50), random_state = 12, max_iter = 1000)

clf.fit(x_train, y_train)

y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)


save_to_pickle(f'{MODEL_FOLDER}/mlp_optimised.pkl', clf, complete_path=False)

In [ ]:
print("Training Accuracy: %.4f" % accuracy_score(y_train, y_train_pred))
print("Testing Accuracy: %.4f" % accuracy_score(y_test, y_test_pred))

### Classification Metrics of the Training Dataset

In [ ]:
classifier_metrics(list(y_train), y_train_pred, print_results = True)

### Classification Metrics of the Testing Dataset

In [ ]:
classifier_metrics(list(y_test), y_test_pred, print_results = True)

### Classification Report of the Testing Dataset

In [ ]:
y_test_pred = clf.predict(x_test)

print(classification_report(y_test, y_test_pred))

### Confusion Matrix of Training Dataset

In [ ]:
import matplotlib.pyplot as plt

predictions = clf.predict(x_train)
cm = confusion_matrix(y_train, predictions, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = clf.classes_)
disp.plot()
plt.show()

### Confusion Matrix of Testing Dataset

In [ ]:
predictions = clf.predict(x_test)
cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = clf.classes_)
disp.plot()
plt.show()

### Loss Curve of MLP Classifier

In [ ]:
plt.plot(clf.loss_curve_)
plt.title("Loss Curve")
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.show()